In [1]:
# Only if you use Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# TODO:
DATA_PATH = '/content/drive/MyDrive/ST_EI/data' 


In [34]:
import pandas as pd
import os
import string
import numpy as np

**Important :**

An Excel file for testing the evaluation part is available in the gitlab repo : evaluation_search_engine_post_queries_ranking_EI_CS.xlsx

If you work on Colab, we advice you to push it directly on your Google Drive directory.

# Evaluate the Search

Now you implement multiple search methods and you're able to improve it. You have to define metric to compare it objectively.



We ask you to implement NDCG (Normalized Discounted Cumulative Gain) from few queries we implement on a dozen of post. We already defined the values of relevance judgement in the xlsx file : . The final score will be the mean quadratic error of the queries.


Explication for the xlsx file :

We propose you a Excel file with some posts and a mesure of relevancy for the queries

- First column is the post Id,
- Columns starting by query are the queries you have to test.
- The values in this columns are the rank of relevancy of the post in regard with the query.
- The missing values indicates you should not take into account the post


You will have to criticize this metric and your result in the report. Then you will have to propose some improvements. 

Thereafter in this week, you will have to compare your different search engines.

In [69]:
posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")

def remove_tags(text:str)->str:
  # TODO
  res = text.replace('<p>',' ').replace('</p>',' ').replace('<\t>',' ').replace('\n',' ').strip()
  return res

clean_posts = posts[['Id','Body']]
clean_posts['cleaned_body'] = clean_posts['Body'].fillna('').apply(remove_tags)


def extract_words(text:str)->list:
  #enlever les majuscules
  text = text.lower()

  #enlever les ponctuations
  text = text.translate(str.maketrans('', '', string.punctuation))

  liste = text.split(' ')
  liste = list(set(liste))
  if '' in liste :
    liste.remove('')

  return liste

clean_posts['words'] = clean_posts['cleaned_body'].apply(extract_words)

<ipython-input-69-6fa7291e6ee7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_posts['cleaned_body'] = clean_posts['Body'].fillna('').apply(remove_tags)
<ipython-input-69-6fa7291e6ee7>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_posts['words'] = clean_posts['cleaned_body'].apply(extract_words)


In [29]:
# Read Relevancy CSV
df_relevancy = pd.read_excel("/content/drive/MyDrive/ST_EI/data/evaluation_search_engine_post_queries_ranking_EI_CS.xlsx")

In [70]:
df = pd.merge(df_relevancy,clean_posts, left_on='PostId', right_on='Id')
df

,PostId,Title,First Sentence,Query 1 : mesure performance for multiclassification model,Query 2 : draw neural network,Query 3 : neural network layers,Query 4 : how sklearn working,Query 5 : treat categorical data,Id,Body,cleaned_body,words
0,6107,What are deconvolutional layers?,I recently read Fully Convolutional Networks f...,NaN,NaN,6.0,NaN,NaN,6107,"<p>I recently read <a href=""http://arxiv.org/a...","I recently read <a href=""http://arxiv.org/abs/...","[called, understanding, 42, lossbr, my, layers..."
1,15989,Micro Average vs Macro average Performance in ...,I am trying out a multiclass classification se...,2.0,NaN,NaN,NaN,NaN,15989,<p>I am trying out a multiclass classification...,I am trying out a multiclass classification se...,"[dataset, equal, codepre, precode, a, distribu..."
2,13490,How to set class weights for imbalanced classe...,I know that there is a possibility in Keras wi...,1.0,NaN,NaN,NaN,NaN,13490,<p>I know that there is a possibility in Keras...,I know that there is a possibility in Keras wi...,"[a, in, would, kind, fitting, weight, find, co..."
3,12321,What's the difference between fit and fit_tran...,I do not understand the difference between the...,NaN,NaN,NaN,7.0,2.0,12321,<p>I do not understand the difference between ...,I do not understand the difference between the...,"[categorical, numbers, codefittransformcode, w..."
4,22,K-Means clustering for mixed numeric and categ...,My data set contains a number of numeric attri...,NaN,NaN,NaN,3.0,3.0,22,<p>My data set contains a number of numeric at...,My data set contains a number of numeric attri...,"[categorical, three, my, iscategoricalattrvalu..."
5,14899,How to draw Deep learning network architecture...,I have built my model.,NaN,4.0,2.0,NaN,NaN,14899,<p>I have built my model. Now I want to draw t...,I have built my model. Now I want to draw the ...,"[architecture, research, hrefhttpsistackimgurc..."
6,5706,"What is the ""dying ReLU"" problem in neural net...",Referring to the Stanford course notes on Conv...,NaN,2.0,5.0,NaN,NaN,5706,<p>Referring to the Stanford course notes on <...,Referring to the Stanford course notes on <a h...,"[happens, proper, your, issue, activate, entir..."
7,15135,Train/Test/Validation Set Splitting in Sklearn,How could I randomly split a data matrix and t...,NaN,NaN,NaN,6.0,NaN,15135,<p>How could I randomly split a data matrix an...,How could I randomly split a data matrix and t...,"[codeytestcode, two, three, a, could, data, co..."
8,12851,How do you visualize neural network architectu...,When writing a paper / making a presentation a...,NaN,5.0,3.0,NaN,NaN,12851,<p>When writing a paper / making a presentatio...,When writing a paper / making a presentation a...,"[architecture, when, a, presentation, networks..."
9,694,Best python library for neural networks,I'm using Neural Networks to solve different M...,NaN,3.0,4.0,5.0,NaN,694,<p>I'm using Neural Networks to solve differen...,I'm using Neural Networks to solve different M...,"[networks, a, in, library, this, and, problems..."


In [24]:
queries = {1:'mesure performance for multiclassification model',2:'draw neural network',3:'neural network layers',4:'how sklearn working',5:'treat categorical data'}

In [43]:
def classement(liste):
  #input : liste
  #output : liste

  classement_parfait = []

  #remplacer les NaN par des 0
  for elem in liste:
    if elem >=0 :
      classement_parfait.append(elem)
    else :
      classement_parfait.append(0)

  return classement_parfait

def dg(x,k):
  return x/np.log2(k+1)


def calculate_ndgc(query_nb, out, df=df, queries=queries):
  # out : liste des docs_id renvoyée par le modèle en ordre décroissant d'importance (index 0 = doc le plus pertinent)
  
  # calcul du dgc du classement parfait
  liste = list(df['Query '+str(query_nb)+' : '+str(queries[query_nb])])
  classement_parfait = classement(liste)
  classement_parfait.sort(reverse = True)
  classement_parfait_dg = [dg(classement_parfait[i],i+1) for i in range (len(classement_parfait))]
  newlist = []
  for i in range(len(classement_parfait_dg)):
    newlist.append(sum(classement_parfait_dg[:i+1]))


  # calcul du dgc du classement du modèle
  classement_model = classement(out)
  classement_model_dg = [dg(classement_model[i],i+1) for i in range (len(classement_model))]
  newlist2 = []
  for i in range(len(classement_model_dg)):
    newlist2.append(sum(classement_model_dg[:i+1]))
  
  # calcul du ndgc
  res = [newlist2[i]/newlist[i] for i in range (len(newlist))]
  

  return res[-1]




# Models


METHODE NAIVE

In [18]:

def word_in_index(word, word_list_index):
  if word in word_list_index:
    res = pd.Series([1])
  else :
    res = pd.Series([0])
  return res

def count_common_words(query, word_serie):
  # TODO
  res = pd.Series([0])
  requete = extract_words(query)
  for word in requete:
    res = res.add(word_in_index(word,word_serie))
  return res

def rank_top_query(query, df, top=5):
  # TODO
  nb_of_common_words = pd.Series([])
  for i in df.index:
    nb_of_common_words = pd.concat([nb_of_common_words,count_common_words(query,df['words'][i])], ignore_index = True)
  df['Common words with query'] = nb_of_common_words
  newdf = df.sort_values(by='Common words with query', ascending = False)
  res = newdf.head(top)
  return res

In [46]:
out1 = list(rank_top_query(query="mesure performance for multiclassification model", df=df, top=12)['Query 1 : mesure performance for multiclassification model'])
out2 = list(rank_top_query(query="draw neural network", df=df, top=12)['Query 2 : draw neural network'])
out3 = list(rank_top_query(query="neural network layers", df=df, top=12)['Query 3 : neural network layers'])
out4 = list(rank_top_query(query="how sklearn working", df=df, top=12)['Query 4 : how sklearn working'])
out5 = list(rank_top_query(query="treat categorical data", df=df, top=12)['Query 5 : treat categorical data'])


ndgc = [calculate_ndgc(1,out1),calculate_ndgc(2,out2),calculate_ndgc(3,out3),calculate_ndgc(4,out4),calculate_ndgc(5,out5)]
print(ndgc)

print(np.mean(ndgc))

[0.3852038879047554, 0.9190820655219575, 0.9561477088560267, 0.6622661640221295, 0.841545187434471]
0.752849002747868


<ipython-input-18-38c03c59c5ee>:32: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  nb_of_common_words = pd.Series([])


METHODE PROBABILISTE

In [47]:
pip install rank-bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
from rank_bm25 import BM25Okapi

In [51]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
liste_stop_words = stopwords.words('english')
print(liste_stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [52]:

def remove_stop_words(l_txt: list) -> list:
  newlist = []
  for word in l_txt:
    if word not in liste_stop_words:
      newlist.append(word)

  return newlist

In [53]:
def extract_words_remove_stop_words(text:str)->list:
  #enlever les majuscules
  text = text.lower()

  #enlever les ponctuations
  text = text.translate(str.maketrans('', '', string.punctuation))

  liste = text.split(' ')
  liste = list(set(liste))
  if '' in liste :
    liste.remove('')

  return remove_stop_words(liste)

def probabilistic_search(query,df):
  # TODO
  corpus = []
  index = []
  for i in df.index:
    corpus.append(df['words'][i])
    index.append(df['Id'][i])
  
  bm25 = BM25Okapi(corpus)

  tokenized_query = extract_words_remove_stop_words(query)

  doc_scores = bm25.get_scores(tokenized_query)

  res = []
  for i in range(len(index)):
    res.append((index[i],doc_scores[i]))
  res.sort(key = lambda x : x[1], reverse = True)
  return res

In [63]:
def traitement_search(search, query_nb):
  liste = pd.Series([])
  for i in range (len(search)):
    liste = pd.concat([liste,df[df['PostId']==search[i][0]]['Query '+str(query_nb)+' : '+str(queries[query_nb])]], ignore_index = True)

  return list(liste)

In [67]:
search1 = traitement_search(probabilistic_search('mesure performance for multiclassification model',df),1)
search2 = traitement_search(probabilistic_search('draw neural network',df),2)
search3 = traitement_search(probabilistic_search('neural network layers',df),3)
search4 = traitement_search(probabilistic_search('how sklearn working',df),4)
search5 = traitement_search(probabilistic_search('treat categorical data',df),5)

ndgc = [calculate_ndgc(1,search1),calculate_ndgc(2,search2),calculate_ndgc(3,search3),calculate_ndgc(4,search4),calculate_ndgc(5,search5)]
print(ndgc)

print(np.mean(ndgc))

[0.8868854556705132, 0.9190820655219575, 0.8360196351516849, 0.5385410308781887, 0.841545187434471]
0.8044146749313631


<ipython-input-63-b01caadd7a07>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  liste = pd.Series([])


METHODE VECTORIELLE

In [72]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [77]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(df.cleaned_body.values)

vectors = vectorizer.transform(df.cleaned_body.values)

In [78]:
vectors

<12x405 sparse matrix of type '<class 'numpy.int64'>'
	with 518 stored elements in Compressed Sparse Row format>

In [75]:
def vectorize_query(query : str, vectorizer=vectorizer):
    """vectorizes the query
    Args:
        query (str): query string
        vectorizer (optional): Defaults to vectorizer.

    Returns:
        query vectorized
    """
    #TODO
    query_vectorized = vectorizer.transform([query]).toarray()

    return query_vectorized

In [85]:

def vectorizer_search(query : str, query_nb,
                      vectors=vectors,
                      vectorizer=vectorizer) -> list:
    #TODO
    vec_query = vectorize_query(query,vectorizer)
    cos_sim = cosine_similarity(vec_query,vectors)[0]

    #créer une liste de tuple (index du doc, similarité avec la requête)
    sim = []
    for i in range(len(cos_sim)):
      sim.append((i,cos_sim[i]))
    sim.sort(key = lambda x : x[1], reverse = True)

    #retrouver les scores du top de similarité
    res = []
    for elem in sim:
      text = df['Query '+str(query_nb)+' : '+str(queries[query_nb])][elem[0]]
      res.append(text)

    return res

In [88]:
entry1 = vectorizer_search('mesure performance for multiclassification model',1)
entry2 = vectorizer_search('draw neural network',2)
entry3 = vectorizer_search('neural network layers',3)
entry4 = vectorizer_search('how sklearn working',4)
entry5 = vectorizer_search('treat categorical data',5)

ndgc = [calculate_ndgc(1,entry1),calculate_ndgc(2,entry2),calculate_ndgc(3,entry3),calculate_ndgc(4,entry4),calculate_ndgc(5,entry5)]
print(ndgc)

print(np.mean(ndgc))

[0.5067916889545789, 0.8502361767883351, 0.9948938993092976, 0.7998507326222158, 0.8310254998828857]
0.7965595995114627
